In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%tensorflow_version 1.x
import keras
from keras.models import  load_model
import pandas as pd
import numpy as np
import os
import cv2

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
df_train = pd.read_csv('/content/drive/My Drive/dance_dataset/train.csv')
len(df_train.shape)

2

In [0]:
def get_y_list(y):
  labels = {'manipuri':0,'bharatanatyam':1,'odissi':2,'kathakali':3,'kathak':4,'sattriya':5,
              'kuchipudi':6,'mohiniyattam':7}
  for i in range(len(y)):
    y[i] = labels[y[i]]
  y = np.expand_dims(y, axis=1)
  return y

In [0]:
def get_data(csv, img_folder):
  df = pd.read_csv(csv)
  img_tensor = []
  # train_list = os.listdir(img_folder)
  train_list = list(df['Image'])
  for img in train_list:
    img_read = cv2.imread(img_folder + '/' + img)
    # print(img, ':', img_read.shape)
    img = cv2.resize(img_read, (224, 224))
    img = img.astype('float32')/255
    img_tensor.append(img)
  # input('stop')
  img_tensor=img_tensor-np.mean(img_tensor, axis=0)
  target = list(df['target'])
  y = get_y_list(target)
  return (img_tensor, y)

In [0]:
train_csv = '/content/drive/My Drive/dance_dataset/train.csv'
training_folder = '/content/drive/My Drive/dance_dataset/train'

In [0]:
# x_train, y_train = get_data(train_csv, training_folder)

In [0]:
# x_train[0].shape

In [0]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Training parameters
batch_size = 34  # orig paper trained all networks with batch_size=128
epochs = 500
data_augmentation = True
num_classes = 8
image_size = 224

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Loading dance dataset
X, Y = get_data(train_csv, training_folder)
x_train, y_train, x_test, y_test = [], [], [], []
for i in range(len(X)):
  if np.random.random()>0.01:
    x_train.append(X[i])
    y_train.append(Y[i])
  else:
    x_test.append(X[i])
    y_test.append(Y[i])

x_train,y_train,x_test,y_test=np.array(x_train),np.array(y_train),np.array(x_test),np.array(y_test)

# Input image dimensions.
input_shape = x_train.shape[1:]

# # Normalize data.
# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255

# # If subtract pixel mean is enabled
# if subtract_pixel_mean:
#     x_train_mean = np.mean(x_train, axis=0)
#     x_train -= x_train_mean
#     x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


from keras.applications import VGG16

#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze all the layers
for layer in vgg_conv.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)


from keras import models
from keras import layers
from keras import optimizers

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
# print(filepath)
# input('stop')
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only = False,
                             mode ='max',
                             period=1)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=40,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.2,
        # set range for random zoom
        zoom_range=0.2,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        steps_per_epoch=len(x_train) / 32,
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (362, 224, 224, 3)
362 train samples
2 test samples
x_test shape: (2, 224, 224, 3)
y_train shape: (362, 1)
y_test shape: (2, 1)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

<keras.engine.input_layer.InputLayer object at 0x7f08dc4bd278> False
<keras.layers.convolutional.Conv2D object at 0x7f08dc4bd518> False
<keras.layers.convolutional.Conv2D object at 0x7f08dc4bd5c0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f08dc01c588> False
<keras.layers.convolutional.Conv2D object at 0x7f08db7b5ac8> False
<keras.layers.convolutional.Conv2D object at 0x7f08db7bf630> False
<keras.layers.pooling.MaxPooling2D object at 0x7f08db7c3828> False
<keras.layers.convolutional.Conv2D object at 0x7f08db7d3898> False
<keras.layers.convolutional.Conv2D object at 0x7f08db7d9400> False
<keras.layers.convolutional.Conv2D object at 0x7f08db7df5f8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f08db7ee1d0> False
<keras.layers.convolutional.Conv2D o

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


12/11 [===============================] - 6s 512ms/step - loss: 1.9954 - accuracy: 0.3384 - val_loss: 0.9904 - val_accuracy: 1.0000
Epoch 3/500
Learning rate:  0.001
12/11 [===============================] - 7s 550ms/step - loss: 1.3056 - accuracy: 0.5304 - val_loss: 1.0141 - val_accuracy: 0.5000
Epoch 4/500
Learning rate:  0.001
12/11 [===============================] - 6s 539ms/step - loss: 1.0739 - accuracy: 0.5981 - val_loss: 0.2660 - val_accuracy: 1.0000
Epoch 5/500
Learning rate:  0.001
12/11 [===============================] - 7s 548ms/step - loss: 0.7775 - accuracy: 0.7279 - val_loss: 0.0954 - val_accuracy: 1.0000
Epoch 6/500
Learning rate:  0.001
12/11 [===============================] - 7s 553ms/step - loss: 0.6715 - accuracy: 0.7790 - val_loss: 0.0162 - val_accuracy: 1.0000
Epoch 7/500
Learning rate:  0.001
12/11 [===============================] - 7s 545ms/step - loss: 0.5076 - accuracy: 0.8301 - val_loss: 0.1170 - val_accuracy: 1.0000
Epoch 8/500
Learning rate:  0.001
12/1

In [0]:
# saving the model
model.save('my_dance_model.h5')

In [0]:
model = load_model("/content/my_dance_model.h5")
# model.summary()

In [0]:
def get_test_data(csv, img_folder):
  df = pd.read_csv(csv)
  img_tensor = []
  # train_list = os.listdir(img_folder)
  train_list = list(df['Image'])
  for img in train_list:
    img_read = cv2.imread(img_folder + '/' + img)
    img = cv2.resize(img_read, (224, 224))
    img = img.astype('float32')/255
    img_tensor.append(img)
  # img_tensor=img_tensor-np.mean(img_tensor, axis=0)
  return np.array(img_tensor)

In [0]:
test_path = "/content/drive/My Drive/dance_dataset/test.csv"
test_folder = "/content/drive/My Drive/dance_dataset/test"
x_test = get_test_data(test_path, test_folder)
# print((df.shape[1]))
# df.head()

In [0]:
def return_output(y):
  labels = {'manipuri':0,'bharatanatyam':1,'odissi':2,'kathakali':3,'kathak':4,'sattriya':5,
              'kuchipudi':6,'mohiniyattam':7}
  for i in range(len(y)):
    for key, val in labels.items():
      if y[i]==val:
        y[i]=key
  return y

In [0]:
# len(y_final)

In [0]:
df = pd.read_csv("/content/drive/My Drive/dance_dataset/test.csv")
y_pred = model.predict(x_test)
y_pred.shape

(156, 8)

In [0]:
y_pred

array([[1.0549998e-04, 2.1127320e-03, 9.9471116e-01, ..., 1.3394020e-06,
        2.9245929e-03, 2.6153937e-06],
       [3.6661862e-07, 1.6977669e-05, 2.7795402e-06, ..., 6.7173853e-05,
        1.0815612e-03, 9.9883085e-01],
       [2.3228285e-08, 5.5680648e-06, 9.9037546e-01, ..., 1.8235795e-06,
        9.6119046e-03, 1.5732888e-07],
       ...,
       [3.4546886e-08, 1.0926777e-06, 9.3310905e-07, ..., 9.8627162e-10,
        4.9970900e-08, 1.9560685e-11],
       [2.8618802e-11, 2.1213140e-09, 9.9999750e-01, ..., 1.0269078e-10,
        2.5360384e-06, 1.4816461e-11],
       [8.1793111e-08, 7.2486822e-11, 2.9102309e-09, ..., 9.9999988e-01,
        1.0625442e-09, 2.8164016e-10]], dtype=float32)

In [0]:
target = []
for i in y_pred:
  target.append(np.argmax(i))
len(target)
print(target)
y_final = return_output(target)

[2, 7, 2, 2, 1, 6, 6, 3, 5, 2, 7, 1, 5, 5, 5, 1, 2, 4, 7, 1, 2, 3, 3, 2, 7, 3, 5, 4, 4, 1, 7, 2, 4, 6, 7, 2, 2, 2, 4, 7, 4, 4, 4, 5, 7, 7, 0, 5, 3, 6, 7, 2, 4, 5, 4, 5, 0, 2, 4, 7, 3, 0, 4, 6, 5, 6, 3, 6, 6, 1, 4, 2, 3, 6, 2, 3, 2, 1, 3, 4, 2, 4, 6, 2, 2, 6, 5, 4, 0, 3, 0, 4, 4, 6, 2, 1, 5, 3, 5, 6, 6, 1, 4, 2, 1, 6, 5, 0, 0, 2, 4, 3, 6, 2, 6, 2, 3, 3, 7, 1, 2, 7, 6, 7, 2, 7, 4, 7, 4, 2, 1, 7, 1, 2, 0, 2, 0, 0, 2, 4, 5, 4, 0, 3, 4, 3, 4, 7, 0, 2, 7, 5, 6, 4, 2, 5]


In [0]:
y_final[0]

'odissi'

In [0]:
df['target'] = y_final
df.to_csv('result.csv', index=False)

In [0]:
from google.colab import files
files.download('/content/result.csv') 